In [1]:
# !pip install pandas	
# !pip install numpy
# !pip install tqdm	
# !pip install matplotlib

In [141]:
# 라이브러리
%load_ext autoreload
%autoreload 2
    
import random
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy.random import randint, rand

import ga_runner
import time
import pickle

SEED_VALUE = 20250526
DIVIDE_VALUE = 5
VERBOSE_VALUE = False
MACHINE_INFO_FILE = "./data/machine_info.csv"
# ORDER_INFO_FILE = "./data/TalkFile_final_orders_completed_ko.csv"
ORDER_INFO_FILE = "./data/order_info_new.csv"
ORDER_INFO_NL_FILE = "./data/order_info_nl.csv"
ORDER_INFO_NL_ENG_FILE = "./data/order_info_nl_eng.csv"
# GT_N5_PKL_FILE = 'gt_n5_augmented_20250618_divide_5.pkl'
GT_N5_PKL_FILE = 'gt_n5_20250618_divide_5_new.pkl'
ORIGIN_GT_N5_FILE = "./dataset_n5_20250526.csv"
# REFACTORY_GT_N5_FILE = "./augmented_dataset_n5_20250618_divide_5.csv"
REFACTORY_GT_N5_FILE = "./gt_n5_20250618_divide_5_new.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
# 데이터 불러오기
machine_info = pd.read_csv(MACHINE_INFO_FILE)
order_info = pd.read_csv(ORDER_INFO_FILE)
order_info_nl = pd.read_csv(ORDER_INFO_NL_FILE, header=None)
order_info_nl_eng = pd.read_csv(ORDER_INFO_NL_ENG_FILE)

In [143]:
order_info_nl.columns = ['영업납기', '중산도면', '단가', '수량', '선급', '자연어주문']

In [144]:
# 전처리 (1): column name 변경
order_info = order_info.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent'
})

machine_info = machine_info.rename(columns={
    'JSDWG': 'item',
    'MCNO': 'machine',
    'AVG_CT': 'capacity'
})

order_info_nl = order_info_nl.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent',
    '자연어주문': 'nl_order'
})

order_info_nl_eng.columns = ['nl_order']

In [145]:
# 머신인포 dropna, 4자리 숫자 문자열로 변환
machine_info.dropna(inplace=True)
machine_info['machine'] = machine_info['machine'].apply(lambda x: str(int(x)).zfill(4))

In [146]:
# 아이템 -> item01
# qty -> 그대로
# 날짜 -> 일수(날짜 - 2021-01-01) date001(1~365) / 자연어는 수정X
# 선급 -> 일단 자연어 그대로
# cost -> 일단 그대로
# machine -> machine01
# capacity -> 일단 그대로

In [147]:
# 아이템 -> item01
unique_items = sorted(order_info['item'].unique(), reverse=True)

items_mapping = {
        item: f"item{str(i+1).zfill(2)}"
        for i, item in enumerate(unique_items)
    }

def change_item_name(target):
    result = str(target)
    for k, v in items_mapping.items():
        result = result.replace(k, v)

    return result

change_item_name('57791')

'57791'

In [148]:
machine_info['item'] = machine_info['item'].apply(change_item_name)
cond = machine_info['item'].isin(items_mapping.values())
machine_info[cond].head()

,item,machine,capacity
897,item96,0408,4.89
898,item96,0409,5.92
899,item96,0410,4.91
900,item96,0412,3.50
901,item96,0416,1.03


In [149]:
order_info['item'] = order_info['item'].apply(change_item_name)
order_info.head()
len(order_info)

127

In [150]:
order_info_nl['item'] = order_info_nl['item'].apply(change_item_name)
order_info_nl['nl_order'] = order_info_nl['nl_order'].apply(change_item_name)
order_info_nl.head()
len(order_info_nl)

127

In [151]:
order_info_nl_eng['nl_order'] = order_info_nl_eng['nl_order'].apply(change_item_name)
order_info_nl_eng.head()

,nl_order
0,"item15, right? 318? You can do it by May 13th,..."
1,"Oh, I just got the item16 drawing, and it says..."
2,"There, I need to do 19 drawings for item96. It..."
3,"Hey, look quickly. item91 4 I have to do it by..."
4,"item87, right? Doing 196? You can do it until ..."


In [152]:
# 날짜 -> 일수(날짜 - 2021-01-01) (1~365)
def change_time_name(target):
    time = pd.to_datetime(target)
    base_date = pd.to_datetime('2021-01-01')

    return f'date{str((time-base_date).days+1).zfill(3)}'

change_time_name('2021-01-01'), change_time_name('2021-12-31'),  change_time_name('2021-03-13')

('date001', 'date365', 'date072')

In [153]:
order_info['time'] = order_info['time'].apply(change_time_name)
order_info.head()

,time,item,cost,qty,urgent
0,date133,item15,25870,318,검사품
1,date133,item16,16229,383,검사품
2,date133,item96,8333,19,NaN
3,date133,item91,36533,4,NaN
4,date133,item87,45500,196,검사품


In [154]:
order_info_nl['time'] = order_info_nl['time'].apply(change_time_name)
order_info_nl.head()

,time,item,cost,qty,urgent,nl_order
0,date133,item15,25870,318,검사품,item15 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 25...
1,date144,item16,16229,383,검사품,헐 방금 item16 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 16...
2,date150,item96,8333,19,NaN,"저기 item96 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. 그리..."
3,date154,item91,36533,4,NaN,어우 빨리 봐봐. item91 4개 21년 6월 3일까지 해야되는데 어떡하지? 단가...
4,date169,item87,45500,196,검사품,item87 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 45...


In [155]:
# machine -> machine01
unique_machines = sorted(machine_info['machine'].dropna().unique())

machines_mapping = {
        machine: f"machine{str(i+1).zfill(2)}"
        for i, machine in enumerate(unique_machines)
    }

def change_machine_name(target):
    result = str(target)
    for k, v in machines_mapping.items():
        result = result.replace(k, v)

    return result

change_machine_name('0421')

'machine19'

In [156]:
machine_info['machine'] = machine_info['machine'].apply(change_machine_name)
machine_info.head()

,item,machine,capacity
0,050060,machine28,1.08
1,050093,machine04,7.13
2,050093,machine08,4.67
3,050093,machine10,4.50
4,050093,machine14,3.92


In [157]:
# 전처리 - order
def preprocess_order(df):
    # 긴급 생산 요건이 있는 경우 1, 그렇지 않은 경우는 0
    df['urgent'] = df['urgent'].fillna(0)

    for i in range(len(df)):
        if df.loc[i, 'urgent'] != 0:
            df.loc[i, 'urgent'] = 1

    df.dropna(axis=0, inplace=True)

    # 같은 종류의 order 묶기
    df = df.groupby(['time','item','cost','urgent']).sum().reset_index()
    # df = df.groupby(['time','item','cost','urgent'], as_index=False).first() 
    
    return df

# 전처리 - machine
def preprocess_machine(df):
    df.dropna(axis=0, inplace=True)

    return df

In [158]:
def make_dataset(sampled_orders, machine_info):
    dataset_for_ga = []
    dataset_for_nl = []
    dataset_for_order = []

    for data in sampled_orders:
        data = data.reset_index(drop=True)  # 여기서 인덱스 재설정
        order = data.iloc[:, :5]
        nl = data.iloc[:, 5:]

        order = preprocess_order(order)
        dataset_for_order.append(order)

        order = pd.merge(order, machine_info, on='item', how='inner')
        dataset_for_ga.append(order)
        dataset_for_nl.append(nl)

    return dataset_for_ga, dataset_for_nl, dataset_for_order


# raw_order_info_nl_eng = order_info_nl_eng.copy()
# 1. 정형 + 자연어 정보 합치기
order_info_nl_eng = pd.concat([order_info, order_info_nl_eng], axis=1)

# 2. 'time' 기준으로 그룹화하여 각 날짜에 해당하는 주문을 하나의 샘플로 저장
sampled_orders = [group.reset_index(drop=True)
                  for _, group in order_info_nl_eng.groupby('time')]

# 3. 'date1', 'date2', ... 순서대로 정렬 (문자열이지만 숫자 기준 정렬)
sampled_orders.sort(key=lambda df: int(df['time'].iloc[0].replace('date', '')))

# 4. 머신 정보 전처리
machine_info = preprocess_machine(machine_info)

# 5. 데이터셋 생성
dataset_for_ga, dataset_for_nl, dataset_for_order = make_dataset(sampled_orders, machine_info)

In [159]:
#전체 묶음 개수 확인
print(f"총 날짜 묶음 수: {len(sampled_orders)}")

# 전체 날짜별 주문 수 확인
order_counts_by_date = order_info_nl_eng.groupby('time').size().reset_index(name='order_count')
display(order_counts_by_date.head(10))  # 상위 10개만 보기


총 날짜 묶음 수: 26


,time,order_count
0,date130,5
1,date133,5
2,date152,5
3,date155,5
4,date161,5
5,date162,5
6,date166,5
7,date168,5
8,date169,5
9,date170,5


### 병렬 처리
---


In [160]:
# 사용할 고정 하이퍼파라미터 
ga_fixed_hyperparams = {
    'n_iter': 500,
    'n_pop': 20,
    'r_cross': 0.4,
    'r_mut': 0.1,
    'max_machine_work_time': 600,
    'overproduction_penalty_factor': 10000000,
    'gene_swap_prob': 0.5 # 우리가 교차 연산자에 추가했던 확률
}
print(f"사용할 고정 하이퍼파라미터: {ga_fixed_hyperparams}")

if 'dataset_for_ga' in globals() and dataset_for_ga: 

    # 병렬 처리 실행 및 시간 측정
    start_time = time.time()
    final_scores, final_gt_list, final_logs = \
        ga_runner.generate_gt_in_parallel(
            dataset_for_ga, 
            ga_fixed_hyperparams,
            output_pickle_path=GT_N5_PKL_FILE,
            verbose=VERBOSE_VALUE
        )
    
    end_time = time.time()

    print(f"\n--- 병렬 처리 실행 요약 ---")
    print(f"총 {len(final_scores)}개의 샘플 처리 완료.")
    print(f"소요 시간: {end_time - start_time:.2f} 초")

    # 처리 결과 확인
    print(f"\n첫 번째 샘플의 Best Score: {final_scores[0] if final_scores else 'N/A'}")
    print("첫 번째 샘플의 GT DataFrame (상위 5개):")
    if final_gt_list and not final_gt_list[0].empty:
        print(final_gt_list[0].head())
    else:
        print("(데이터 없음)")

else:
    print("dataset_for_ga가 준비되지 않아 실행할 수 없습니다.")


사용할 고정 하이퍼파라미터: {'n_iter': 500, 'n_pop': 20, 'r_cross': 0.4, 'r_mut': 0.1, 'max_machine_work_time': 600, 'overproduction_penalty_factor': 10000000, 'gene_swap_prob': 0.5}
병렬 처리에 사용할 CPU 코어(프로세스) 수: 22

총 26개의 샘플에 대해 병렬 처리를 시작합니다...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 26.14it/s]

모든 샘플에 대한 병렬 처리 완료!

최종 GT 리스트 저장 완료: gt_n5_20250618_divide_5_new.pkl

--- 병렬 처리 실행 요약 ---
총 26개의 샘플 처리 완료.
소요 시간: 1.16 초

첫 번째 샘플의 Best Score: 0
첫 번째 샘플의 GT DataFrame (상위 5개):
     item    machine     time  qty
0  item05  machine10  date130  398
1  item06  machine22  date130   18
2  item12  machine08  date130    1
3  item12  machine09  date130    3
4  item12  machine14  date130    3


In [161]:
with open(GT_N5_PKL_FILE, 'rb') as f:
    refactored_gt_ls = pickle.load(f)

refactored_gt_ls[10]

,item,machine,time,qty
0,item04,machine04,date173,6
1,item04,machine14,date173,6
2,item04,machine28,date173,2
3,item08,machine08,date173,9
4,item08,machine09,date173,3
5,item08,machine10,date173,4
6,item08,machine14,date173,7
7,item08,machine21,date173,7
8,item08,machine22,date173,10
9,item08,machine31,date173,8


In [162]:
# GT 결과값 데이테프레임화
# for idx, gt in enumerate(refactored_gt_ls):
#     gt[['item', 'machine', 'time']] = pd.DataFrame(gt['(item, machine, time)'].tolist(), index=gt.index)
#     gt = gt.drop(columns='(item, machine, time)')
#     gt = gt[['item', 'machine', 'time', 'qty']].sort_values(['time', 'item', 'machine', 'qty'])
#     refactored_gt_ls[idx] = gt

refactored_gt_ls[0]

,item,machine,time,qty
0,item05,machine10,date130,398
1,item06,machine22,date130,18
2,item12,machine08,date130,1
3,item12,machine09,date130,3
4,item12,machine14,date130,3
5,item12,machine21,date130,3
6,item12,machine22,date130,1
7,item12,machine23,date130,1
8,item51,machine08,date130,26
9,item51,machine09,date130,93


In [163]:
# order 정보 묶기
def combine_order_nl(data):
    combined_data = '\n'.join(f'• {text}' for text in data['nl_order'])

    return combined_data

combine_order_nl(dataset_for_nl[0])

"• item05 You know? I have to do 398 of them, and it's until May 10th, 2021. It's 2,990 won, nothing special.\n• item51 came. 208 need to be done and it's due on May 13th, 2021. Unit price 18,500 won. Oh, right, inspection items.\n• Look at this. They want 15 item12s, but they say it's until May 30th, 2021. 15,640 won, nothing special.\n• There's item95 that came yesterday. I have to do two of those, and it's until May 30th, 2021. It's 20,000 won, and there's nothing special about it.\n• I want 150 of these item06. They have to be done by May 31st, 2021, and the unit price is 4,000 won. Oh, and there's nothing special about it."

In [164]:
order_nl_for_dataset = [combine_order_nl(data) for data in dataset_for_nl]
order_nl_for_dataset[0]

"• item05 You know? I have to do 398 of them, and it's until May 10th, 2021. It's 2,990 won, nothing special.\n• item51 came. 208 need to be done and it's due on May 13th, 2021. Unit price 18,500 won. Oh, right, inspection items.\n• Look at this. They want 15 item12s, but they say it's until May 30th, 2021. 15,640 won, nothing special.\n• There's item95 that came yesterday. I have to do two of those, and it's until May 30th, 2021. It's 20,000 won, and there's nothing special about it.\n• I want 150 of these item06. They have to be done by May 31st, 2021, and the unit price is 4,000 won. Oh, and there's nothing special about it."

In [165]:
dataset_for_order[0]

,time,item,cost,urgent,qty
0,date130,item05,2990,0,398
1,date130,item06,4000,0,150
2,date130,item12,15640,0,15
3,date130,item51,18500,1,208
4,date130,item95,20000,0,2


In [166]:
dataset_for_ga[0]

,time,item,cost,urgent,qty,machine,capacity
0,date130,item05,2990,0,398,machine10,4.60
1,date130,item06,4000,0,150,machine22,1.32
2,date130,item12,15640,0,15,machine08,8.17
3,date130,item12,15640,0,15,machine09,10.62
4,date130,item12,15640,0,15,machine14,8.22
5,date130,item12,15640,0,15,machine21,9.32
6,date130,item12,15640,0,15,machine22,12.50
7,date130,item12,15640,0,15,machine23,11.21
8,date130,item51,18500,1,208,machine08,4.59
9,date130,item51,18500,1,208,machine09,6.02


In [167]:
# order machine join 정보 묶기
def combine_order_machine_join(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['time']} {row['item']} {row['cost']} {row['urgent']} {row['qty']} {row['machine']} {row['capacity']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

combine_order_machine_join(dataset_for_ga[0])

'• date130 item05 2990 0 398 machine10 4.6\n• date130 item06 4000 0 150 machine22 1.32\n• date130 item12 15640 0 15 machine08 8.17\n• date130 item12 15640 0 15 machine09 10.62\n• date130 item12 15640 0 15 machine14 8.22\n• date130 item12 15640 0 15 machine21 9.32\n• date130 item12 15640 0 15 machine22 12.5\n• date130 item12 15640 0 15 machine23 11.21\n• date130 item51 18500 1 208 machine08 4.59\n• date130 item51 18500 1 208 machine09 6.02\n• date130 item51 18500 1 208 machine29 4.17\n• date130 item95 20000 0 2 machine10 25.0\n• date130 item95 20000 0 2 machine19 200.0'

In [168]:
order_machine_join = [combine_order_machine_join(data) for data in dataset_for_ga]
print(order_machine_join[0])

• date130 item05 2990 0 398 machine10 4.6
• date130 item06 4000 0 150 machine22 1.32
• date130 item12 15640 0 15 machine08 8.17
• date130 item12 15640 0 15 machine09 10.62
• date130 item12 15640 0 15 machine14 8.22
• date130 item12 15640 0 15 machine21 9.32
• date130 item12 15640 0 15 machine22 12.5
• date130 item12 15640 0 15 machine23 11.21
• date130 item51 18500 1 208 machine08 4.59
• date130 item51 18500 1 208 machine09 6.02
• date130 item51 18500 1 208 machine29 4.17
• date130 item95 20000 0 2 machine10 25.0
• date130 item95 20000 0 2 machine19 200.0


In [169]:
def combine_order(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['time']} {row['item']} {row['cost']} {row['urgent']} {row['qty']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_order(dataset_for_order[0]))

• date130 item05 2990 0 398
• date130 item06 4000 0 150
• date130 item12 15640 0 15
• date130 item51 18500 1 208
• date130 item95 20000 0 2


In [170]:
order_for_dataset = [combine_order(data) for data in dataset_for_ga]
print(f"원본 리스트 길이: {len(order_for_dataset)}")
order_for_dataset[0]


원본 리스트 길이: 26


'• date130 item05 2990 0 398\n• date130 item06 4000 0 150\n• date130 item12 15640 0 15\n• date130 item12 15640 0 15\n• date130 item12 15640 0 15\n• date130 item12 15640 0 15\n• date130 item12 15640 0 15\n• date130 item12 15640 0 15\n• date130 item51 18500 1 208\n• date130 item51 18500 1 208\n• date130 item51 18500 1 208\n• date130 item95 20000 0 2\n• date130 item95 20000 0 2'

In [171]:
def combine_machine(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['item']} {row['machine']} {row['capacity']}")

    combined_data = list(set(combined_data))
    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_machine(dataset_for_ga[0]))

• item12 machine21 9.32
• item51 machine08 4.59
• item12 machine09 10.62
• item05 machine10 4.6
• item95 machine10 25.0
• item12 machine23 11.21
• item51 machine29 4.17
• item12 machine22 12.5
• item95 machine19 200.0
• item51 machine09 6.02
• item12 machine08 8.17
• item12 machine14 8.22
• item06 machine22 1.32


In [172]:
machine_for_dataset = [combine_machine(data) for data in dataset_for_ga]
machine_for_dataset[0]

'• item12 machine21 9.32\n• item51 machine08 4.59\n• item12 machine09 10.62\n• item05 machine10 4.6\n• item95 machine10 25.0\n• item12 machine23 11.21\n• item51 machine29 4.17\n• item12 machine22 12.5\n• item95 machine19 200.0\n• item51 machine09 6.02\n• item12 machine08 8.17\n• item12 machine14 8.22\n• item06 machine22 1.32'

In [173]:
# gt 정보 텍스트화
def combine_gt(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['item']} {row['machine']} {row['time']} {row['qty']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_gt(refactored_gt_ls[0]))

• item05 machine10 date130 398
• item06 machine22 date130 18
• item12 machine08 date130 1
• item12 machine09 date130 3
• item12 machine14 date130 3
• item12 machine21 date130 3
• item12 machine22 date130 1
• item12 machine23 date130 1
• item51 machine08 date130 26
• item51 machine09 date130 93
• item51 machine29 date130 89
• item95 machine19 date130 2


In [174]:
refactored_gt_for_dataset = [combine_gt(data) for data in refactored_gt_ls]
refactored_gt_for_dataset[0]

'• item05 machine10 date130 398\n• item06 machine22 date130 18\n• item12 machine08 date130 1\n• item12 machine09 date130 3\n• item12 machine14 date130 3\n• item12 machine21 date130 3\n• item12 machine22 date130 1\n• item12 machine23 date130 1\n• item51 machine08 date130 26\n• item51 machine09 date130 93\n• item51 machine29 date130 89\n• item95 machine19 date130 2'

In [175]:
refactored_dataset = pd.DataFrame(columns=['order', 'nl', 'machine', 'gt', 'objective'])
refactored_dataset

,order,nl,machine,gt,objective


In [176]:
print(f"final_scores 길이: {len(final_scores)}")

final_scores 길이: 26


In [177]:
refactored_dataset['order'] = order_for_dataset
refactored_dataset['nl'] = order_nl_for_dataset
refactored_dataset['machine'] = machine_for_dataset
refactored_dataset['gt'] = refactored_gt_for_dataset
refactored_dataset['objective'] = final_scores
refactored_dataset.head()

,order,nl,machine,gt,objective
0,• date130 item05 2990 0 398\n• date130 item06 ...,"• item05 You know? I have to do 398 of them, a...",• item12 machine21 9.32\n• item51 machine08 4....,• item05 machine10 date130 398\n• item06 machi...,0
1,• date133 item15 25870 1 318\n• date133 item15...,"• item15, right? 318? You can do it by May 13t...",• item16 machine07 4.85\n• item15 machine33 4....,• item15 machine04 date133 25\n• item15 machin...,0
2,• date152 item57 366680 0 2\n• date152 item57 ...,• I just got an urgent call for item89. I have...,• item72 machine10 3.04\n• item57 machine09 12...,• item72 machine04 date152 2\n• item72 machine...,0
3,• date155 item15 25870 1 318\n• date155 item15...,"• Wait, check this. Is this drawing item35 28?...",• item78 machine33 5.18\n• item15 machine33 4....,• item15 machine04 date155 12\n• item15 machin...,0
4,• date161 item10 5050 0 178\n• date161 item10 ...,"• Hey, look at this, is that 178 item10? It's ...",• item18 machine22 3.41\n• item59 machine08 3....,• item10 machine07 date161 11\n• item10 machin...,0


In [178]:
refactored_dataset['nl_machine'] = (
    "Order information\n" + refactored_dataset['nl'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine']
)

refactored_dataset['nl_machine_gt'] = (
    "Order information\n" + refactored_dataset['nl'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine'] + "\n\n" +
    "Schedule result\n" + refactored_dataset['gt']
)

refactored_dataset['order_machine'] = (
    "Order information\n" + refactored_dataset['order'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine']
)

refactored_dataset['order_machine_gt'] = (
    "Order information\n" + refactored_dataset['order'] + "\n\n" +
    "Machine information\n" + refactored_dataset['machine'] + "\n\n" +
    "Schedule result\n" + refactored_dataset['gt']
)

refactored_dataset['order_machine_join'] = order_machine_join

refactored_dataset['order_machine_join_gt'] = (
    "Order and machine information\n" + refactored_dataset['order_machine_join'] + "\n\n" +
    "Schedule result\n" + refactored_dataset['gt']
)

refactored_dataset.head()

,order,nl,machine,gt,objective,nl_machine,nl_machine_gt,order_machine,order_machine_gt,order_machine_join,order_machine_join_gt
0,• date130 item05 2990 0 398\n• date130 item06 ...,"• item05 You know? I have to do 398 of them, a...",• item12 machine21 9.32\n• item51 machine08 4....,• item05 machine10 date130 398\n• item06 machi...,0,Order information\n• item05 You know? I have t...,Order information\n• item05 You know? I have t...,Order information\n• date130 item05 2990 0 398...,Order information\n• date130 item05 2990 0 398...,• date130 item05 2990 0 398 machine10 4.6\n• d...,Order and machine information\n• date130 item0...
1,• date133 item15 25870 1 318\n• date133 item15...,"• item15, right? 318? You can do it by May 13t...",• item16 machine07 4.85\n• item15 machine33 4....,• item15 machine04 date133 25\n• item15 machin...,0,"Order information\n• item15, right? 318? You c...","Order information\n• item15, right? 318? You c...",Order information\n• date133 item15 25870 1 31...,Order information\n• date133 item15 25870 1 31...,• date133 item15 25870 1 318 machine04 12.84\n...,Order and machine information\n• date133 item1...
2,• date152 item57 366680 0 2\n• date152 item57 ...,• I just got an urgent call for item89. I have...,• item72 machine10 3.04\n• item57 machine09 12...,• item72 machine04 date152 2\n• item72 machine...,0,Order information\n• I just got an urgent call...,Order information\n• I just got an urgent call...,Order information\n• date152 item57 366680 0 2...,Order information\n• date152 item57 366680 0 2...,• date152 item57 366680 0 2 machine04 12.0\n• ...,Order and machine information\n• date152 item5...
3,• date155 item15 25870 1 318\n• date155 item15...,"• Wait, check this. Is this drawing item35 28?...",• item78 machine33 5.18\n• item15 machine33 4....,• item15 machine04 date155 12\n• item15 machin...,0,"Order information\n• Wait, check this. Is this...","Order information\n• Wait, check this. Is this...",Order information\n• date155 item15 25870 1 31...,Order information\n• date155 item15 25870 1 31...,• date155 item15 25870 1 318 machine04 12.84\n...,Order and machine information\n• date155 item1...
4,• date161 item10 5050 0 178\n• date161 item10 ...,"• Hey, look at this, is that 178 item10? It's ...",• item18 machine22 3.41\n• item59 machine08 3....,• item10 machine07 date161 11\n• item10 machin...,0,"Order information\n• Hey, look at this, is tha...","Order information\n• Hey, look at this, is tha...",Order information\n• date161 item10 5050 0 178...,Order information\n• date161 item10 5050 0 178...,• date161 item10 5050 0 178 machine07 1.83\n• ...,Order and machine information\n• date161 item1...


In [179]:
print(refactored_dataset.iloc[0, -3])

Order information
• date130 item05 2990 0 398
• date130 item06 4000 0 150
• date130 item12 15640 0 15
• date130 item12 15640 0 15
• date130 item12 15640 0 15
• date130 item12 15640 0 15
• date130 item12 15640 0 15
• date130 item12 15640 0 15
• date130 item51 18500 1 208
• date130 item51 18500 1 208
• date130 item51 18500 1 208
• date130 item95 20000 0 2
• date130 item95 20000 0 2

Machine information
• item12 machine21 9.32
• item51 machine08 4.59
• item12 machine09 10.62
• item05 machine10 4.6
• item95 machine10 25.0
• item12 machine23 11.21
• item51 machine29 4.17
• item12 machine22 12.5
• item95 machine19 200.0
• item51 machine09 6.02
• item12 machine08 8.17
• item12 machine14 8.22
• item06 machine22 1.32

Schedule result
• item05 machine10 date130 398
• item06 machine22 date130 18
• item12 machine08 date130 1
• item12 machine09 date130 3
• item12 machine14 date130 3
• item12 machine21 date130 3
• item12 machine22 date130 1
• item12 machine23 date130 1
• item51 machine08 date130 26


In [180]:
refactored_dataset.to_csv(REFACTORY_GT_N5_FILE)

**규범님이 order 컬럼에 동일 데이터가 중복 되어 토근 낭비 된다고하여 중복 제거**

In [181]:
def remove_duplicate_lines(multiline_string):
    """
    하나의 긴 문자열(여러 줄로 구성)을 입력받아,
    내부의 중복된 라인을 제거하고 순서를 유지한 새로운 문자열을 반환합니다.
    """
    if not isinstance(multiline_string, str):
        return multiline_string
    
    lines = multiline_string.split('\n')
    # 혹시 모를 빈 라인 항목은 제거
    non_empty_lines = [line for line in lines if line.strip()]
    unique_lines = list(dict.fromkeys(non_empty_lines))
    return '\n'.join(unique_lines)


try:
    df = pd.read_csv(REFACTORY_GT_N5_FILE)
    print(f"'{REFACTORY_GT_N5_FILE}' 파일 로드 성공. 총 {len(df)}개 행.")

    df['order_unique'] = df['order'].apply(remove_duplicate_lines)

    # 기존 'order' 컬럼을 삭제
    df = df.drop(columns=['order'])
    
    # 새로 만든 'order_unique' 컬럼의 이름을 'order'로 변경
    df = df.rename(columns={'order_unique': 'order'})
    
    # 컬럼 순서를 사용자가 원하는 대로 재정렬합니다.
    print("\n'order' 컬럼을 두 번째 위치로 재정렬합니다...")
    
    all_columns = df.columns.tolist()
    
    # 'order' 컬럼을 리스트에서 일단 제거합니다.
    all_columns.remove('order')
    
    # 리스트의 두 번째 위치(인덱스 1)에 'order' 컬럼 이름을 다시 삽입합니다.
    all_columns.insert(1, 'order')
    
    # 이 새로운 순서 리스트를 사용해서 DataFrame의 컬럼 순서를 재정의합니다.
    df = df[all_columns]

    # 7. 수정된 DataFrame을 새로운 CSV 파일로 저장
    #    (원본 파일을 덮어쓰지 않도록 새로운 파일 이름을 사용)
    output_filename = REFACTORY_GT_N5_FILE.replace('.csv', '_deduplicated.csv')
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print(f"\n중복 라인이 제거된 데이터가 '{output_filename}' 파일로 저장되었습니다.")
    print("\n--- 저장된 DataFrame (상위 5개) ---")
    print(df.head())

except FileNotFoundError:
    print(f"오류: '{REFACTORY_GT_N5_FILE}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")

'./gt_n5_20250618_divide_5_new.csv' 파일 로드 성공. 총 26개 행.

'order' 컬럼을 두 번째 위치로 재정렬합니다...

중복 라인이 제거된 데이터가 './gt_n5_20250618_divide_5_new_deduplicated.csv' 파일로 저장되었습니다.

--- 저장된 DataFrame (상위 5개) ---
   Unnamed: 0                                              order  \
0           0  • date130 item05 2990 0 398\n• date130 item06 ...   
1           1  • date133 item15 25870 1 318\n• date133 item16...   
2           2  • date152 item57 366680 0 2\n• date152 item72 ...   
3           3  • date155 item15 25870 1 318\n• date155 item35...   
4           4  • date161 item10 5050 0 178\n• date161 item18 ...   

                                                  nl  \
0  • item05 You know? I have to do 398 of them, a...   
1  • item15, right? 318? You can do it by May 13t...   
2  • I just got an urgent call for item89. I have...   
3  • Wait, check this. Is this drawing item35 28?...   
4  • Hey, look at this, is that 178 item10? It's ...   

                                             machine  \
0